# Модель нейронной сети для предсказания сбоя в работе авиационного двигателя

## Краткая информация

**Введение**   
В этом блокноте разрабатывается модель  для прогнозирования технического обслуживания авиационных двигателей. Основная цель - предсказать остаточный срок службы этих двигателей на основе показаний датчиков и эксплуатационных параметров.  
**Dataset¶**  
Набор данных, используемый в этом проекте, состоит из показаний датчиков и настроек двигателя, собранных за определенное время. 

## Установка библеотек

In [212]:
#!pip install numpy
#!pip install pandas
#!pip install sklearn

In [217]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import accuracy_score, classification_report
import pickle

## Подготовка данны

### Загрузка данных

In [181]:
df_train = pd.read_excel('~\\PM_train.xlsx')
df_test = pd.read_excel("~\\PM_test.xlsx")
df_truth = pd.read_excel('~\\PM_truth.xlsx')


### Объедение df_train и df_test для работы со всем набором данных

In [184]:
df = pd.concat([df_train, df_test], ignore_index=True)
df.head()


,id,cycle,setting1,setting2,setting3,s1,s2,s3,s4,s5,...,s12,s13,s14,s15,s16,s17,s18,s19,s20,s21
0,1,1,-0.0007,-0.0004,100,518.67,641.82,1589.70,1400.60,14.62,...,521.66,2388.02,8138.62,8.4195,0.03,392.0,2388,100,39.06,23.4190
1,1,2,0.0019,-0.0003,100,518.67,642.15,1591.82,1403.14,14.62,...,522.28,2388.07,8131.49,8.4318,0.03,392.0,2388,100,39.00,23.4236
2,1,3,-0.0043,0.0003,100,518.67,642.35,1587.99,1404.20,14.62,...,522.42,2388.03,8133.23,8.4178,0.03,390.0,2388,100,38.95,23.3442
3,1,4,0.0007,0.0000,100,518.67,642.35,1582.79,1401.87,14.62,...,522.86,2388.08,8133.83,8.3682,0.03,392.0,2388,100,38.88,23.3739
4,1,5,-0.0019,-0.0002,100,518.67,642.37,1582.85,1406.22,14.62,...,522.19,2388.04,8133.80,8.4294,0.03,393.0,2388,100,38.90,23.4044


### Ввод новой переменной

Оставшийся срок службы (RUL) - это важнейшая концепция прогнозируемого технического обслуживания, представляющая собой расчетный срок службы или количество циклов, оставшихся у актива до того момента, когда он должен выйти из строя. В контексте авиационных двигателей RUL помогает предвидеть момент, когда двигатель может перестать отвечать требованиям по производительности. Рассчитав RUL, организации могут заблаговременно планировать мероприятия по техническому обслуживанию, оптимизировать распределение ресурсов и минимизировать время простоя. Такой прогностический подход позволяет своевременно принимать меры, снижая риск неожиданных отказов и повышая общую эффективность и надежность активов.

* Remaining Useful Life (RUL): рассчитывается путем вычитания текущего цикла из максимального цикла для каждого двигателя.
* Feature Columns: список столбцов признаков для обучения модели.


In [192]:
df['RUL'] = df.groupby('id')['cycle'].transform(max) - df['cycle']
feature_columns = ['setting1', 'setting2', 'setting3', 's1', 's2', 's3', 's4', 's5', 's6', 's7', 's8', 's9', 's10', 's11', 's12', 
                   's13', 's14', 's15', 's16', 's17', 's18', 's19', 's20', 's21']

C:\Users\User\AppData\Local\Temp\ipykernel_1592\3293406284.py:1: FutureWarning: The provided callable <built-in function max> is currently using SeriesGroupBy.max. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "max" instead.
  df['RUL'] = df.groupby('id')['cycle'].transform(max) - df['cycle']


### Объединение истинных данных  
Объединение данных df с истинными данными (df_truth) на основе идентификатора двигателя   
позволяет получить дополнительную информацию для анализа.

In [196]:

df = pd.merge(df, df_truth, on='id', how='left')

### Labeling

In [199]:
threshold = 30
df['label'] = (df['RUL'] <= threshold).astype(int)

### Разделение df на тестовую и тренеровочную выборку

In [230]:

X_train, X_test, y_train, y_test = train_test_split(df[feature_columns], df['RUL'], test_size=0.2, random_state=42)
X_test.to_excel('data.xlsx', index=False)

## Построение модели

In [204]:
model = LinearRegression()
model.fit(X_train, y_train)

LinearRegression()

## Прогнозирование и оценка

In [233]:
y_pred = model.predict(X_test)

y_test_binary = (y_test <= threshold).astype(int)
y_pred_binary = (y_pred <= threshold).astype(int)

accuracy = accuracy_score(y_test_binary, y_pred_binary)
print(f'Accuracy: {accuracy}')
print(classification_report(y_test_binary, y_pred_binary))

Accuracy: 0.9455019957015659
              precision    recall  f1-score   support

           0       0.95      1.00      0.97      5883
           1       0.91      0.48      0.63       631

    accuracy                           0.95      6514
   macro avg       0.93      0.74      0.80      6514
weighted avg       0.94      0.95      0.94      6514



## Сохранение модели в формате pickle

In [221]:
with open('model.pkl', 'wb') as file:
    pickle.dump(model, file)